In [ ]:
import pandas as pd
from pyprojroot import here

import matplotlib.pyplot as plt

In [ ]:
path = 'data/raw/pirha-capa-export/tulva-capacity-data_2018-01-01_-_2020-12-31.csv'
data = pd.read_csv(here() / path, 
                   delimiter=';', 
                   names=['Node key', 'Identifier', 'Capacity', 'Datetime'],
                   parse_dates=True)

df = pd.pivot_table(data, values='Capacity', index='Datetime', columns='Node key')


In [ ]:
df.index = pd.to_datetime(df.index, format='mixed', utc=True);
df = df.resample('h').last().fillna(method='ffill');


In [ ]:
df.index = df.index.tz_localize(None)

In [ ]:
path = 'data/raw/pirha-capa-export/tulva-node-key_2018-01-01_-_2020-12-31.csv'
node_keys = pd.read_csv(here() / path, 
                        delimiter=';',
                        names=['Node key', 'Name', 'Short Name'])

d = node_keys.set_index('Node key')['Name'].to_dict()

In [ ]:
columns = [d[x] for x in df.columns]
df.columns = columns

In [ ]:
# Fix PTO anomaly
df.loc[df['ACUT/PTO'] > 30, 'ACUT/PTO'] = None

In [ ]:
df.to_csv(here() / 'data/extra/beds-all.csv')

In [ ]:
facilities = ['ACUT/PTO',
              'Akaa tk',
              'HA2',
              'HA3',
              'HA4',
              'HA5',
              'HASA B5',
              'HASA U3',
              'HB3',
              'HB4 (Puistosair.)',
              'HINF',
              'HSO1',
              'Hämeenkyrön tk',
              'Ikaalisten tk',
              'Kangasalan tk',
              'Lempäälän tk',
              'Nokian tk',
              'Nokian tk / Lempäälä',
              'Pirkkalan tk',
              'Urjala tk',
              'Valkeakoski tk',
              'Ylöjärven tk']

In [ ]:
df = df[facilities]

In [ ]:
columns = [f'Beds-{x}' for x in df.columns]
df.columns = columns

In [ ]:
df['Beds-All'] = df.sum(axis=1)

In [ ]:
df.to_csv(here() / 'data/preprocessed/beds.csv')